In [1]:
import re
import requests
import urllib.request
from urllib.parse import urljoin

from bs4 import BeautifulSoup

import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_hdf('../metadata.h5')
df.head(50)

,children_folders,children_urls,date_added,date_modified,guid,id,meta_info,name,parents,type,url,keyword,img_url
0,"[1884, 1891, 1897, 1928, 1936, 1958, 1980, 161...","[1903, 1906, 1907, 1908, 1909, 1910, 1911, 191...",13202479594197706,13242831177794171,00000000-0000-4000-a000-000000000002,1,,書籤列,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",folder,,[],None
1,[],"[1885, 1886, 1888, 1890]",13233003241034750,13233003241037911,f0b52077-6793-498a-b8eb-462dfc400922,1884,,2019-nCoV,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",folder,,"[2019, nCoV]",None
2,[],[],13226238181043741,,7f8a21b9-b559-44ab-b711-c41800d188d1,1885,,Coronavirus_Taiwan,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",url,https://viator.maps.arcgis.com/apps/opsdashboa...,"[Coronavirus, Taiwan]",None
3,[],[],13226407607803828,,24a7d7be-2a87-4481-b157-3773db2f4eab,1886,,2019-nCoV | 武漢肺炎地圖,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",url,https://kiang.github.io/2019-nCoV/,"[2019, nCoV, 武漢, 地圖, 肺炎]",None
4,[],[],13227360115873492,,a09ae3f0-3a5f-417c-ad10-423d35e155f3,1888,,程序員硬核勸告：現在還不是出門的時候 - 幫趣,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",url,https://bangqu.com/a7wXH5.html,"[硬核, 勸告, 現在還, 出門, 時候, 幫趣, 程序]",None
5,[],[],13232695304731928,,8c555a82-3c7b-4f14-8f19-9cfe663e5d19,1890,,300新竹市東區高翠路173巷4弄 - Google 地圖,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",url,https://www.google.com/maps/place/300%E6%96%B0...,"[300, 東區, 高翠路, 173, Google, 地圖, 新竹市]",None
6,[],"[1892, 1893, 1894, 1896, 2819, 2843, 2862]",13233003241047126,13242239034086184,90b08b84-66d4-488d-8e18-20cdf3ba014a,1891,,iiNumbers,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",folder,,[iiNumbers],None
7,[],[],13228386784256522,,8d719a89-0635-42d2-a750-52ab77bd8d8f,1892,,Hubstaff - My account: Billing,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",url,https://app.hubstaff.com/users/796279/billing,"[Hubstaff, My, account, Billing]",None
8,[],[],13228386905254476,,ab84fcbe-ce9f-4823-b200-9066e72d3896,1893,,[HQ] New Member On Board,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",url,https://3.basecamp.com/3287947/projects/4147783,"[HQ, New, Member, Board]",None
9,[],[],13230802944793614,,b66208e6-5322-4eac-9841-bb50844182ee,1894,,iiNumbers HQ / Business Development Office / D...,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",url,https://gitlab.iinumbers.net/iinumbers-hq/bdo/...,"[Project, iiNumbers, HQ, Business, Development...",None


In [3]:
def get_thumbnail(row):
    """去找網站裡面已經預設好的 thumbnail，結果會有下面三種情形：
    
        1. None: 沒有結果，之後會重跑
        2. '': 有連到，但判定為沒有 thumbnail
        3. 'http:...': 成功找到 thumbnail 網址
    """

    if not row['url']:
        return ''

    if row['img_url'] == None:
        try:
            with requests.get(row['url'], timeout=1.5) as res:
                html = BeautifulSoup(res.text, 'html.parser')
            img_node = html.find('meta', attrs={"property":"og:image"})
            if img_node:
                return img_node.get('content')
            else:
                return ''
        except Exception as e:
            return None

for i, row in tqdm(list(df.iterrows())):
    df.loc[i, 'img_url'] = get_thumbnail(row)

100%|████████████████████████████████████████████████████████████████████████████████| 919/919 [18:12<00:00,  1.05s/it]


In [4]:
df.head()

,children_folders,children_urls,date_added,date_modified,guid,id,meta_info,name,parents,type,url,keyword,img_url
0,"[1884, 1891, 1897, 1928, 1936, 1958, 1980, 161...","[1903, 1906, 1907, 1908, 1909, 1910, 1911, 191...",13202479594197706,13242831177794171,00000000-0000-4000-a000-000000000002,1,,書籤列,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",folder,,[],
1,[],"[1885, 1886, 1888, 1890]",13233003241034750,13233003241037911,f0b52077-6793-498a-b8eb-462dfc400922,1884,,2019-nCoV,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",folder,,"[2019, nCoV]",
2,[],[],13226238181043741,,7f8a21b9-b559-44ab-b711-c41800d188d1,1885,,Coronavirus_Taiwan,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",url,https://viator.maps.arcgis.com/apps/opsdashboa...,"[Coronavirus, Taiwan]",
3,[],[],13226407607803828,,24a7d7be-2a87-4481-b157-3773db2f4eab,1886,,2019-nCoV | 武漢肺炎地圖,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",url,https://kiang.github.io/2019-nCoV/,"[2019, nCoV, 武漢, 地圖, 肺炎]",http://kiang.github.io/2019-nCoV/og_image.png
4,[],[],13227360115873492,,a09ae3f0-3a5f-417c-ad10-423d35e155f3,1888,,程序員硬核勸告：現在還不是出門的時候 - 幫趣,"[書籤列, 2019-nCoV, Coronavirus_Taiwan, 2019-nCoV...",url,https://bangqu.com/a7wXH5.html,"[硬核, 勸告, 現在還, 出門, 時候, 幫趣, 程序]",https://d2ndd3gtcc6iwc.cloudfront.net/liang/20...


In [5]:
df.to_hdf('../metadata.h5', key='data')

C:\Users\adam8\Anaconda3\lib\site-packages\pandas\core\generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['children_folders', 'children_urls', 'date_added', 'date_modified', 'guid', 'id', 'meta_info', 'name', 'parents', 'type', 'url', 'keyword', 'img_url']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [6]:
def get_image(row):
    if row['img_url']:
        img_url = urljoin(*row[['url', 'img_url']])

        try:
            urllib.request.urlretrieve(img_url, f"../static/images/{row['guid']}")
                
        except Exception as e:
            print(e)

df.progress_apply(get_image, axis='columns')

  3%|██▌                                                                              | 29/919 [00:07<05:02,  2.94it/s]

HTTP Error 403: Forbidden


  4%|██▉                                                                              | 33/919 [00:08<04:39,  3.17it/s]

HTTP Error 403: Forbidden


  7%|█████▋                                                                           | 64/919 [00:13<02:47,  5.10it/s]

...

100%|███████████████████████████████████████████████████████████████████████████████▋| 916/919 [03:02<00:00,  5.02it/s]

HTTP Error 403: Forbidden


100%|████████████████████████████████████████████████████████████████████████████████| 919/919 [03:02<00:00,  4.43it/s]

HTTP Error 403: Forbidden


0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
       ... 
889    None
890    None
891    None
892    None
893    None
894    None
895    None
896    None
897    None
898    None
899    None
900    None
901    None
902    None
903    None
904    None
905    None
906    None
907    None
908    None
909    None
910    None
911    None
912    None
913    None
914    None
915    None
916    None
917    None
918    None
Length: 919, dtype: object